In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
base_url = 'https://www.bbc.com/sport/football/premier-league/top-scorers'

In [24]:
player_names = []
team_names = []
goals = []
assists = []
num_matches = []
shots = []

In [38]:
try:
    response = requests.get(base_url)
    response.raise_for_status()
except requests.exceptions.HTTPError as err:
    raise SystemExit(err)
else:
    status_code = response.status_code
    print(f'Status code: {status_code}')
    if status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        players = soup.find('tbody').find_all('tr', class_='ssrcss-dhlz6k-TableRowBody e1icz100')
        for player in players:
            player_name = player.find('div', class_ = "ssrcss-m6ah29-PlayerName e1n8xy5b1").get_text(strip=True)
            team_name = player.find('div', class_ = "ssrcss-qvpga1-TeamsSummary e1n8xy5b0").get_text(strip=True)
            goals_scored = player.find('div', class_ = "ssrcss-8k20kk-CellWrapper ef9ipf0").get_text(strip=True)

            stats = player.find_all('div', class_ = "ssrcss-150z8d-CellWrapper ef9ipf0")
            assists_made = int(stats[0].get_text(strip=True))
            matches_played = int(stats[2].get_text(strip=True))
            shots_taken = int(stats[-3].get_text(strip=True))
            
            player_names.append(player_name)
            team_names.append(team_name)
            goals.append(goals_scored)
            assists.append(assists_made)
            num_matches.append(matches_played)
            shots.append(shots_taken)

        data = {
            'player': player_names,
            'team': team_names,
            'matches': num_matches,
            'goals': goals,
            'assists': assists,
            'shots': shots
        }
        df_players = pd.DataFrame(data)
    else:
        print('Failed to fetch data')
finally:
    print('Done')

Status code: 200
Done


In [39]:
df_players.to_csv('top_scorers.csv', index=False)

In [40]:
df_players.head()

,player,team,matches,goals,assists,shots
0,Mohamed Salah,Liverpool,29,27,17,106
1,E. Haaland,Man City,28,21,3,102
2,A. Isak,Newcastle,25,19,5,69
3,C. Wood,Nottm Forest,29,18,3,53
4,B. Mbeumo,Brentford,29,15,5,62


In [37]:
df_players.describe()

,matches,assists,shots
count,39.000000,39.000000,39.000000
mean,26.358974,4.128205,58.512821
std,2.590338,2.820783,19.177662
min,20.000000,0.000000,31.000000
25%,25.500000,3.000000,44.500000
50%,27.000000,4.000000,53.000000
75%,28.000000,5.000000,64.500000
max,29.000000,17.000000,106.000000
